In [5]:
import logging
import json
import yaml
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.client_credential import ClientCredential
import os

In [42]:
# If running in a notebook, manually set the path or use os.getcwd()
config_folder = os.path.join(os.getcwd(), '../config')

# Paths to the configuration and credentials files
config_path = os.path.join(config_folder, 'config.yaml')
credentials_path = os.path.join(config_folder, 'credentials.json')


In [43]:
# Load the configuration files
def load_config():
    with open(config_path, 'r') as file:
        return yaml.safe_load(file)

def load_credentials():
    with open(credentials_path, 'r') as file:
        return json.load(file)

In [45]:
# Load config and credentials
config = load_config()
credentials = load_credentials()

In [46]:
# Get SharePoint URL and credentials from the loaded files
SHAREPOINT_URL = config["sharepoint"]["site_url"]
CLIENT_ID = credentials["client_id"]
CLIENT_SECRET = credentials["client_secret"]

In [47]:
# Create ClientCredential instance
client_credential = ClientCredential(CLIENT_ID, CLIENT_SECRET)

In [48]:
# Initialize ClientContext with authentication
ctx = ClientContext(SHAREPOINT_URL).with_credentials(client_credential)


In [50]:
# Function to upload a file
def upload_file(folder_url, file_path):
    """Uploads a file to the specified SharePoint folder."""
    try:
        with open(file_path, "rb") as file_content:
            folder = ctx.web.get_folder_by_server_relative_url(folder_url)
            folder.upload_file(file_path.split("/")[-1], file_content)
            ctx.execute_query()
            logging.info(f"Uploaded file: {file_path}")
            print(f"Uploaded: {file_path}")
    except Exception as e:
        logging.error(f"Upload failed: {e}")
        print(f"Upload failed: {e}")


In [51]:
# Function to download a file
def download_file(file_url, save_path):
    """Downloads a file from SharePoint."""
    try:
        file = ctx.web.get_file_by_server_relative_url(file_url)
        content = file.download()
        ctx.execute_query()
        
        with open(save_path, "wb") as f:
            f.write(content.read())
        
        logging.info(f"Downloaded file: {file_url}")
        print(f"Downloaded: {save_path}")
    except Exception as e:
        logging.error(f"Download failed: {e}")
        print(f"Download failed: {e}")


In [52]:
# Function to delete a file
def delete_file(file_url):
    """Deletes a file from SharePoint."""
    try:
        file = ctx.web.get_file_by_server_relative_url(file_url)
        file.delete_object()
        ctx.execute_query()
        logging.info(f"Deleted file: {file_url}")
        print(f"Deleted: {file_url}")
    except Exception as e:
        logging.error(f"Delete failed: {e}")
        print(f"Delete failed: {e}")


In [53]:
# Function to list files in a SharePoint folder
def list_files(folder_url):
    """Lists files in a SharePoint folder."""
    try:
        folder = ctx.web.get_folder_by_server_relative_url(folder_url)
        files = folder.files.get().execute_query()
        file_list = [file.properties["Name"] for file in files]
        logging.info(f"Listed files in {folder_url}: {file_list}")
        print(f"Files in {folder_url}: {file_list}")
    except Exception as e:
        logging.error(f"List failed: {e}")
        print(f"List files failed: {e}")


In [54]:
# Example usage in a notebook

# Upload a file
upload_file("Shared Documents/RealEstate", "/path/to/your/local/file.xlsx")

# Download a file
download_file("/sites/yoursite/Shared Documents/RealEstate/yourfile.xlsx", "/path/to/save/yourfile.xlsx")

# Delete a file
delete_file("/sites/yoursite/Shared Documents/RealEstate/yourfile.xlsx")

# List files in a folder
list_files("Shared Documents/RealEstate")

[18/Mar/2025 15:45:37] ERROR - Upload failed: [Errno 2] No such file or directory: '/path/to/your/local/file.xlsx'
Upload failed: [Errno 2] No such file or directory: '/path/to/your/local/file.xlsx'
[18/Mar/2025 15:45:37] ERROR - Download failed: download() missing 1 required positional argument: 'file_object'
Download failed: download() missing 1 required positional argument: 'file_object'
[18/Mar/2025 15:45:37] ERROR - Delete failed: Acquire app-only access token failed.
Delete failed: Acquire app-only access token failed.
[18/Mar/2025 15:45:37] ERROR - List failed: Acquire app-only access token failed.
List files failed: Acquire app-only access token failed.
